In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="Q&A_over_CSV"

In [2]:
import pandas as pd

df = pd.read_csv("titanic.csv")
print(df.shape)
print(df.columns.tolist())

(891, 12)
['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


# SQL

You can store the Pandas dataframe in a SQLLiteDB.

Using SQL to interact with CSV data is the recommended approach because it is easier to limit permissions and sanitize queries than with arbitrary Python.

Most SQL databases make it easy to load a CSV file in as a table (DuckDB, SQLite, etc.). Once you've done this you can use all of the chain and agent-creating techniques outlined in the SQL tutorial. Here's a quick example of how we might do this with SQLite:


Once this is done - you can create an SQL agent or apply the chains that we've prepared in folder "SQL_Q&A_Chain"

In [3]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic", engine, index=False)

891

In [4]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM titanic WHERE Age < 2;"))

sqlite
['titanic']
[(79, 1, 2, 'Caldwell, Master. Alden Gates', 'male', 0.83, 0, 2, '248738', 29.0, None, 'S'), (165, 0, 3, 'Panula, Master. Eino Viljami', 'male', 1.0, 4, 1, '3101295', 39.6875, None, 'S'), (173, 1, 3, 'Johnson, Miss. Eleanor Ileen', 'female', 1.0, 1, 1, '347742', 11.1333, None, 'S'), (184, 1, 2, 'Becker, Master. Richard F', 'male', 1.0, 2, 1, '230136', 39.0, 'F4', 'S'), (306, 1, 1, 'Allison, Master. Hudson Trevor', 'male', 0.92, 1, 2, '113781', 151.55, 'C22 C26', 'S'), (382, 1, 3, 'Nakid, Miss. Maria ("Mary")', 'female', 1.0, 0, 2, '2653', 15.7417, None, 'C'), (387, 0, 3, 'Goodwin, Master. Sidney Leonard', 'male', 1.0, 5, 2, 'CA 2144', 46.9, None, 'S'), (470, 1, 3, 'Baclini, Miss. Helene Barbara', 'female', 0.75, 2, 1, '2666', 19.2583, None, 'C'), (645, 1, 3, 'Baclini, Miss. Eugenie', 'female', 0.75, 2, 1, '2666', 19.2583, None, 'C'), (756, 1, 2, 'Hamalainen, Master. Viljo', 'male', 0.67, 1, 1, '250649', 14.5, None, 'S'), (789, 1, 3, 'Dean, Master. Bertram Vere', 'mal

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


# Initialize the agent 

We'll use the SQLDatabaseToolkit to create a bunch of tools:

* Create and execute queries
* Check query syntax
* Retrieve table descriptions
  
... and more

In [6]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()
tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000026E881878E0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000026E881878E0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000026E881878E0>),
 QuerySQLCheckerTool(description='Use this tool to 

# System Prompt
We will also want to create a system prompt for our agent. This will consist of instructions for how to behave.

In [7]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""


In [8]:
system_message = SystemMessage(content=SQL_PREFIX)

# Initializing agent
First, get required package LangGraph


We will use a prebuilt LangGraph agent to build our agent



In [9]:
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)

C:\Users\neural_ninja\AppData\Local\Temp\ipykernel_9672\2704392570.py:4: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(llm, tools, messages_modifier=system_message)


In [11]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="what's the average age of survivors?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_78SA4pkwfizT49jH8OwLQ9tB', 'function': {'arguments': '{}', 'name': 'sql_db_list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 549, 'total_tokens': 561}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a31de210-9745-458a-9c62-96676f83a4fd-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'call_78SA4pkwfizT49jH8OwLQ9tB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 549, 'output_tokens': 12, 'total_tokens': 561})]}}
----
{'tools': {'messages': [ToolMessage(content='titanic', name='sql_db_list_tables', tool_call_id='call_78SA4pkwfizT49jH8OwLQ9tB')]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Uf13dgxoW74Zq6w44LteELCL', 'function': {'arguments': '{"table

# Printing only the final output

In [12]:
def print_final_generation(agent_executor,user_input):
    last_agent_message = None

    # Stream the output from the agent executor
    for s in agent_executor.stream(
        {"messages": [HumanMessage(content=user_input)]}
    ):
        # Check if the message is from the agent and contains content
        if 'agent' in s and 'messages' in s['agent']:
            for message in s['agent']['messages']:
                if isinstance(message, AIMessage) and message.content:
                    last_agent_message = message.content

    # Print the last agent message
    if last_agent_message:
        print(last_agent_message)

In [13]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    print_final_generation(agent_executor,user_input)

Here are some examples of the correlation between age and fare from the Titanic dataset:

1. Age: 0.42 years, Fare: 8.5167
2. Age: 0.67 years, Fare: 14.5
3. Age: 0.75 years, Fare: 19.2583
4. Age: 0.75 years, Fare: 19.2583
5. Age: 0.83 years, Fare: 29.0

These entries show that as age increases (in years), the fare also tends to increase, though the sample size is very limited. To draw stronger conclusions, a larger dataset would be required to analyze the correlation more effectively.
Goodbye!
